# Encode text data

In [1]:
import pickle
import numpy as np
import pandas as pd

In [2]:
with open("preprocessed_questions.pkl", "rb") as f:
    preprocessed_questions = pickle.load(f)

In [3]:
mask = preprocessed_questions["text_length_lemma"] > 0
preprocessed_questions = preprocessed_questions.loc[mask, :]

## Bag of Words

In [4]:
from sklearn.feature_extraction.text import CountVectorizer

In [5]:
bow_vectorizer = CountVectorizer()
bow_encoded_questions = bow_vectorizer.fit_transform(preprocessed_questions['clean_text_lemma'])

In [ ]:
with open("bow_embeddings.pkl", "wb") as f:
    pickle.dump(bow_encoded_questions, f)

## TF-IDF

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [7]:
tfidf_vectorizer = TfidfVectorizer()
tfidf_encoded_questions = tfidf_vectorizer.fit_transform(preprocessed_questions['clean_text_lemma'])

In [25]:
with open("tfidf_embeddings.pkl", "wb") as f:
    pickle.dump(tfidf_encoded_questions, f)

## Word2Vec

In [8]:
from gensim.models import Word2Vec

In [9]:
tokenised_sentences = preprocessed_questions["clean_text_lemma"].apply(lambda sent: sent.split()).to_dict()

In [15]:
w2v_model = Word2Vec(list(tokenised_sentences.values()), vector_size=300, min_count=1)

In [16]:
def get_document_vector(doc, wv):
    """
    Remove out-of-vocabulary words.
    Create document vectors by averaging word vectors.
    """
    vocab_tokens = [word for word in doc if word in wv.index_to_key]
    return np.mean(wv.__getitem__(vocab_tokens), axis=0)

In [20]:
w2v_encoded_questions = {k: get_document_vector(v, w2v_model.wv) for k, v in tokenised_sentences.items()}

In [22]:
with open("w2v_embeddings.pkl", "wb") as f:
    pickle.dump(w2v_encoded_questions, f)

## Glove

In [26]:
import gensim
import gensim.downloader as api
from gensim.models import KeyedVectors
print(list(gensim.downloader.info()['models'].keys()))

['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']


In [31]:
wv.save()

TypeError: save() missing 1 required positional argument: 'fname_or_handle'

In [29]:
wv = api.load('glove-twitter-200')                    # load Embeddings (download)
wv.save('.embeddings/200d_glove_vec.kv')               # Save Embeddings to disk - Approx 800MB Size

# wv = KeyedVectors.load('embeddings/200d_glove_vec.kv')  # Load Embeddings (from disk)

FileNotFoundError: [Errno 2] No such file or directory: 'embeddings/200d_glove_vec.kv.vectors.npy'